In [14]:
import numpy as np
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

In [15]:
from graph_attention_layer import GraphAttention
from  untils import load_data,preprocess_features

In [16]:
# Read data
A, X, Y_train, Y_val, Y_test, idx_train, idx_val, idx_test = load_data('cora')

In [17]:
print('X的shape:',X.shape)
print('A的shape:',A.shape)
print('Y_train的shape:',Y_train.shape)

X的shape: (2708, 1433)
A的shape: (2708, 2708)
Y_train的shape: (2708, 7)


In [18]:
# Parameters
N = X.shape[0]                # Number of nodes in the graph
F = X.shape[1]                # Original feature dimension
n_classes = Y_train.shape[1]  #标签的维度
F_ = 8                        # Output size of first GraphAttention layer
n_attn_heads = 8              # Number of attention heads  即K值
dropout_rate = 0.6            # Dropout rate (between and inside GAT layers)
l2_reg = 5e-4/2
learning_rate = 5e-3
epochs = 1000
es_patience = 100


In [19]:
# 对行进行规范化
X= preprocess_features(X)

In [20]:
X.shape

(2708, 1433)

In [21]:
A = A +np.eye(A.shape[0])  #加上自连接

In [22]:
## 定义模型输入
X_in = Input(shape=(F,))
A_in = Input(shape=(N,))

dropout1 = Dropout(dropout_rate)(X_in)

#第一次注意力层
graph_attention_1 = GraphAttention(F_,
                                  attn_heads=n_attn_heads,
                                  dropout_rate=dropout_rate,
                                  activation='elu',
                                  kernel_regularizer=l2(l2_reg),
                                  attn_kernel_regularizer=l2(l2_reg))([dropout1, A_in])


dropout2 = Dropout(dropout_rate)(graph_attention_1)

#第二次注意力层
graph_attention_2 = GraphAttention(n_classes,
                                  attn_heads=1,
                                  attn_heads_reduction='average',
                                  dropout_rate=dropout_rate,
                                  activation='softmax',
                                  kernel_regularizer=l2(l2_reg),
                                  attn_kernel_regularizer=l2(l2_reg))([dropout2, A_in])

model = Model(inputs=[X_in, A_in], outputs=graph_attention_2)

optimizer = Adam(lr=learning_rate)
model.compile(optimizer = optimizer,
             loss='categorical_crossentropy',
             weighted_metrics=['acc'])

model.summary()

# Callbacks
es_callback = EarlyStopping(monitor='val_weighted_acc', patience=es_patience)
# tb_callback = TensorBoard(batch_size=N)
mc_callback = ModelCheckpoint('logs/best_model.h5',
                              monitor='val_weighted_acc',
                              save_best_only=True,
                              save_weights_only=True)



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_21 (Dropout)            (None, 1433)         0           input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
graph_attention_3 (GraphAttenti (None, 64)           91904       dropout_21[0][0]                 
                                                                 input_4[0][0]              

In [23]:
# Train model
validation_data = ([X,A],Y_val,idx_val)
model.fit([X,A],
         Y_train,
        sample_weight=idx_train,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[es_callback,mc_callback])
# callbacks=[es_callback,tb_callback,mc_callback]
# Test model
eval_results = model.evaluate([X,A],
                             Y_test,
                             sample_weight=idx_test,
                             batch_size=N,
                             verbose=0)

print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

Train on 2708 samples, validate on 2708 samples
Epoch 1/1000
2708/2708 [==============================] - 2s 657us/step - loss: 0.1436 - acc: 0.1786 - val_loss: 0.3967 - val_acc: 0.1420
Epoch 2/1000


D:\learn software\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_weighted_acc` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
D:\learn software\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_weighted_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


2708/2708 [==============================] - 0s 157us/step - loss: 0.1379 - acc: 0.1071 - val_loss: 0.3913 - val_acc: 0.1780
Epoch 3/1000
2708/2708 [==============================] - 0s 155us/step - loss: 0.1324 - acc: 0.2000 - val_loss: 0.3865 - val_acc: 0.1420
Epoch 4/1000
2708/2708 [==============================] - 0s 160us/step - loss: 0.1281 - acc: 0.1571 - val_loss: 0.3823 - val_acc: 0.1600
Epoch 5/1000
2708/2708 [==============================] - 0s 172us/step - loss: 0.1240 - acc: 0.1929 - val_loss: 0.3786 - val_acc: 0.1440
Epoch 6/1000
2708/2708 [==============================] - 0s 157us/step - loss: 0.1205 - acc: 0.1929 - val_loss: 0.3755 - val_acc: 0.1480
Epoch 7/1000
2708/2708 [==============================] - 0s 169us/step - loss: 0.1182 - acc: 0.1571 - val_loss: 0.3731 - val_acc: 0.1520
Epoch 8/1000
2708/2708 [==============================] - 0s 156us/step - loss: 0.1158 - acc: 0.1857 - val_loss: 0.3711 - val_acc: 0.1640
Epoch 9/1000
2708/2708 [=======================

2708/2708 [==============================] - 0s 169us/step - loss: 0.1035 - acc: 0.1857 - val_loss: 0.3613 - val_acc: 0.1480
Epoch 62/1000
2708/2708 [==============================] - 0s 168us/step - loss: 0.1034 - acc: 0.2286 - val_loss: 0.3613 - val_acc: 0.1500
Epoch 63/1000
2708/2708 [==============================] - 0s 166us/step - loss: 0.1034 - acc: 0.2429 - val_loss: 0.3613 - val_acc: 0.2020
Epoch 64/1000
2708/2708 [==============================] - 0s 166us/step - loss: 0.1033 - acc: 0.2429 - val_loss: 0.3613 - val_acc: 0.4020
Epoch 65/1000
2708/2708 [==============================] - 0s 168us/step - loss: 0.1031 - acc: 0.2786 - val_loss: 0.3613 - val_acc: 0.4180
Epoch 66/1000
2708/2708 [==============================] - 0s 171us/step - loss: 0.1034 - acc: 0.1857 - val_loss: 0.3613 - val_acc: 0.3020
Epoch 67/1000
2708/2708 [==============================] - 0s 170us/step - loss: 0.1032 - acc: 0.2286 - val_loss: 0.3613 - val_acc: 0.1320
Epoch 68/1000
2708/2708 [================

Epoch 120/1000
2708/2708 [==============================] - 0s 173us/step - loss: 0.1015 - acc: 0.1786 - val_loss: 0.3597 - val_acc: 0.1840
Epoch 121/1000
2708/2708 [==============================] - 0s 171us/step - loss: 0.1016 - acc: 0.1571 - val_loss: 0.3598 - val_acc: 0.1840
Epoch 122/1000
2708/2708 [==============================] - 0s 172us/step - loss: 0.1015 - acc: 0.2357 - val_loss: 0.3598 - val_acc: 0.1900
Epoch 123/1000
2708/2708 [==============================] - 0s 170us/step - loss: 0.1015 - acc: 0.1500 - val_loss: 0.3598 - val_acc: 0.2020
Epoch 124/1000
2708/2708 [==============================] - 0s 171us/step - loss: 0.1014 - acc: 0.1857 - val_loss: 0.3599 - val_acc: 0.2100
Epoch 125/1000
2708/2708 [==============================] - 0s 170us/step - loss: 0.1012 - acc: 0.2071 - val_loss: 0.3598 - val_acc: 0.2560
Epoch 126/1000
2708/2708 [==============================] - 0s 169us/step - loss: 0.1013 - acc: 0.2214 - val_loss: 0.3597 - val_acc: 0.3180
Epoch 127/1000
2708/

Epoch 179/1000
2708/2708 [==============================] - 0s 172us/step - loss: 0.1008 - acc: 0.2000 - val_loss: 0.3594 - val_acc: 0.1940
Epoch 180/1000
2708/2708 [==============================] - 0s 169us/step - loss: 0.1008 - acc: 0.2214 - val_loss: 0.3594 - val_acc: 0.1820
Epoch 181/1000
2708/2708 [==============================] - 0s 166us/step - loss: 0.1009 - acc: 0.1857 - val_loss: 0.3594 - val_acc: 0.1880
Epoch 182/1000
2708/2708 [==============================] - 0s 165us/step - loss: 0.1008 - acc: 0.1857 - val_loss: 0.3594 - val_acc: 0.2060
Epoch 183/1000
2708/2708 [==============================] - 0s 167us/step - loss: 0.1009 - acc: 0.2000 - val_loss: 0.3594 - val_acc: 0.2200
Epoch 184/1000
2708/2708 [==============================] - 0s 166us/step - loss: 0.1010 - acc: 0.1929 - val_loss: 0.3593 - val_acc: 0.2280
Epoch 185/1000
2708/2708 [==============================] - 0s 168us/step - loss: 0.1010 - acc: 0.1571 - val_loss: 0.3594 - val_acc: 0.2340
Epoch 186/1000
2708/

KeyboardInterrupt: 